# 📘 Semi-Implicit PIC Solver in Electromagnetic Fields


This project implements a **semi-implicit Particle-in-Cell (PIC) solver** for simulating
particle motion in **electric** and **magnetic fields**, inspired by the formulations from
**Vu & Brackbill (1982)** and **Lapenta (2006)**.



# 🧩 Core Algorithm Steps

## Initialization
- Simulation Paramter: 3D
- Border: 25 x 25 x 25
- Grid Points: 10 x 10 x 10
- Particle per Cell(NPpCell): 20
- Implicit : Theta 0.5 <br/>
### Grid
- Grid is saved as Mesh (X,Y,Z) np.meshgrid   so you get Xpositon with X[x][y][z] <br/>
- The Cmd :"Indexing i,j" Is for the Order of Dimension x,y,z  if you choose x,y it whould switches bc it just does [arrayx*y]*z  
- Grid Point Has Fields
- Every Grid Point 1 Electric Field ($\theta$) and Magnetic Field
- Also density $\rho$ <br/>

#### Computational Particel
- Group Of particles it Represent(Constant)
- with Custum Shape Function Exists between the Grid Points
- Velocity,Force,Elekctric Field,Magnetic Field <br/>
- Are all 3 Dimensional First Index (x,y,z) <br/>
- Second Index Particel Index  Total=(NPpCell*VolumeGridpoints(100)) <br/>
### CFl Condition
$$
\begin{align}
  & \varepsilon < v_{\text{th}} \frac{\Delta t}{\Delta x} < 1 \quad \text{mit } v_{\text{th}} = c \\
  & \Delta x \sim \lambda_p = \sqrt{ \frac{\varepsilon_0 k_B T}{n_e e^2} }  \quad \quad
   v_{\text{th}} = \sqrt{ \frac{k_B T}{m} }  \quad \quad
   \Delta t \sim \omega_{pe}^{-1} = \sqrt{ \frac{\varepsilon_0 m}{n e^2} } \\
  &f(v) = \sqrt{\frac{m}{2\pi k_B T}} \exp\left( -\frac{m v^2}{2 k_B T} \right)
\end{align}
$$

## Final Result: 
**To keep in Mind the Operator mu or better the Shapefunction transports from Particle Space to Grid Space. Operator R stays in the Particle Space** <br/>
<br/>
$$
\begin{aligned}
&\mu (\vec{k}) = 4\pi \theta\Delta t \sum_p q_p  \beta_p R_p(\vec{k}) S\left(\mathbf{x}-\mathbf{x}^n\right) \\ 
&\hat{\rho}^n=\rho^n-\Delta t \theta \nabla \cdot \hat{\mathbf{J}}^n \\
& R_p(\vec{k}) =\frac{\vec{k}+\frac{\beta_p}{ c} \vec{k} \times \mathbf{B}_p^n+\frac{\beta^2_p}{ c^2}\left(\vec{k} \cdot \mathbf{B}_p^n\right) \mathbf{B}_p^n}{1+\frac{\beta_p^2}{ c^2}\left|B_p^n\right|^2}\\
&\hat{\mathbf{J}}^n=\sum_p q_p R_p\left(\mathbf{v}_p\right)  S\left(\mathbf{x}-\mathbf{x}^n\right)-\theta \Delta t \nabla \cdot \sum_p q_p |R_p\left(\mathbf{v}_p\right)|^2  S\left(\mathbf{x}-\mathbf{x}^n\right)
\end{aligned}
$$ 
<br/>
$$
\begin{aligned}
& = \mathbf{E}^{n+\theta} + \mu( \mathbf{E}^{n+\theta})
- (c\theta\Delta t)^2 
\left[
\nabla^2 \mathbf{E}^{n+\theta}
+ \nabla \nabla \cdot (\mu \cdot \mathbf{E}^{n+\theta})
\right] 
\\[2ex]
 & =
\mathbf{E}^n
+ c\theta\Delta t \left( \nabla \times \mathbf{B}^n - \frac{4\pi}{c} \hat{\mathbf{J}}^n \right)
- (c\theta\Delta t)^2 4\pi \nabla \hat{\rho}^n \\
\end{aligned}
$$
### This Forms this Matrix Equation
$$
\begin{aligned}
\mathbf{A} \vec{E}^{n+\theta} =\vec{b}
\end{aligned}
$$


# Maxwell Equation
$$
\begin{aligned}
&\text{Gauss's law for electricity:} && \nabla \cdot \mathbf{E} = 4\pi \rho \\
&\text{Gauss's law for magnetism:} && \nabla \cdot \mathbf{B} = 0 \\
&\text{Faraday's law of induction:} && \nabla \times \mathbf{E} = -\frac{1}{c} \frac{\partial \mathbf{B}}{\partial t} \\
&\text{Ampère's law (with Maxwell's correction):} && \nabla \times \mathbf{B} = \frac{1}{c} \frac{\partial \mathbf{E}}{\partial t} + \frac{4\pi}{c} \mathbf{J}
\end{aligned}
$$


# Developing Step with Loop j
At the Moments is the Implemenation that only the Field to Particle gets Iterative updated <br/>
unclear if it should iterate over the Shaperfunction. <br/>
**Step 0  <br/>
$(x_p^{n})_{j=0}=x_p^{n}$** <br/>
**$(v_p^{n})_{j=0}=v_p^{n}$**
## Moment Gathering:
Partikel to Grid
$$
\rho^{n}_i = \sum_p q_p S\!\left(\mathbf{x}_i - (\mathbf{x_p}^{n})_{j}\right)
$$
Shaper Function berechnet ein Gewicht bassierend auf dem Abstand zu den nächsten Gridpoints. <br/>
Außerdem berechenn von
- $\hat{J} \left( (v_p^{n})_{j=0} , (x_p^{n})_{j=0} \right)$
- $\hat{\rho}$
Damit erhält man $\vec{b}$
$$
\begin{aligned}
\vec{b} & =\mathbf{E}^n + c\theta\Delta t \left( \nabla \times \mathbf{B}^n - \frac{4\pi}{c} \hat{\mathbf{J}}^n \right) - (c\theta\Delta t)^2 4\pi \nabla \hat{\rho}^n
\end{aligned}
$$
### Solve with this $E^{n+\theta}$ GMRES Method
**This Method Includes Evolvded Particle Density $\rho$ and Currents J**<br/> 
Convert Fields to Particle:
$$
\begin{aligned}
& \hat{\mathbf{v}}_p = \mathbf{v}_p^n + \frac{q_p \Delta t}{2 m_p} \mathbf{E}_p^{n+\theta}((x_p^{n})_{j}) \\[1em]
& (\mathbf{v}_p)_{j} = R \;\bigl(\hat{\mathbf{v}}_p\bigr) \\[1em]
& (x_p^{n})_{j+1} = x_p^{n} + \frac12 \Delta t \, (v_p^{n})_{j}
\end{aligned}
$$
# After Itereation K Resolve
$$
\begin{aligned}
& \mathbf{B}^{n+1} = \mathbf{B}^n - c \Delta t \nabla \times \mathbf{E}^{n+\theta} \\
& \mathbf{E}^{n+1} = \frac{\mathbf{E}^{n+\theta} - (1-\theta) \mathbf{E}^n}{\theta} \\
& \text{mit } \theta = 0.5 \\
& \mathbf{E}^{n+1} = 2\mathbf{E}^{n+\theta} - \mathbf{E}^n \\
& \mathbf{v}^{n+1} = 2\mathbf{v}^{n+\theta} - \mathbf{v}^n \\
& \mathbf{x}^{n+1} = \mathbf{x}_p^n + \mathbf{v}^{n+\theta} \Delta t
\end{aligned}
$$

**Code Implenetation Side Node** <br/>


# Math Operation used During Calc. for $E^{n+\theta}$

$$
\begin{aligned}
A \cdot \mathbf{E}^{n+\theta} 
&= (\mathbf{I} + \mu) \cdot \mathbf{E}^{n+\theta} 
- (c\theta\Delta t)^2 
\bigl[ \nabla^2 \mathbf{E}^{n+\theta} 
+ \nabla \nabla \cdot (\mu \cdot \mathbf{E}^{n+\theta}) \bigr]
\end{aligned}
$$

### Implementierung aller Ableitungen
- $\nabla^2$ (Laplace) Vektorfeld $F$: $F_i$ nach $x^2,y^2,z^2$ abgeleitet
- $\nabla \nabla$ (Grad(Div($\boldsymbol{\cdot}$))) Summe aus part. Ableitungen
- $\nabla \times$ (Curl)
- $\nabla$ (Divergence)

**Notiz:**
$$
\text{grad}(\text{div}(\mathbf{F})) 
= \nabla (\nabla \cdot \mathbf{F}) 
= \left( 
\frac{\partial}{\partial x} (\nabla \cdot \mathbf{F}), 
\frac{\partial}{\partial y} (\nabla \cdot \mathbf{F}), 
\frac{\partial}{\partial z} (\nabla \cdot \mathbf{F})
\right)
$$




## Deposition From Particles to Grid and vise versa for Later Fields
- **Iter** Particle <br/>
    - Take Pos X,Y,Z Devide my d(x,y,z)
    - Floor it (int())
    - Calcalate all 8 Volume if +1  Volume Distribution and Add them to Gridpoints (-1)^m*(x-xp)*(y-yp)*(z-zp) **Wrong Potentially**
    -  do 1d each and multiplay easier
    -  use Reaptive MApping Modulo <br/>

From Code PIC II ion background to neutralize:                   **Not Implemented** <br/>
- rho -= NP/mx 
- rho *= 2*NPpCell*charge/dx
**Also for Current**




**Sanity Check (Energy Balance):**
- $\mathbf{Electric Energy} \quad 0.5 \sum (Efield^2)*dx$
- $\mathbf{Total Engery}$
- $\mathbf{Kinetic Energy} \quad 0.5*\rho_{weight} \sum (vp^2)*dx$ <br/>
**t=t+dt**

In [5]:
"""
There Is A Line Break to See.      (╯°□°)╯︵ ┻━┻
"""
print()

# Derivation Intermediate Velocity
### Particle Mover (Semi-Implicit)  in CGS: Gauss System (SI without c) c =29,979,245,800 cm/s
Update particle positions f is Placeholder for Future Particle Position:
        $$
          \vec{x}^{n+1} = \vec{x}^n + \mathbf{v}_f \; \Delta t
        $$
Update particle velocities with a semi-implicit scheme. Here is Half Step Derivative:
$$ \begin{align}
             \frac{\vec{v}^{n+1} - \vec{v}^n}{\Delta t} = \frac{q}{m}\left( \vec{E}^{n+\theta}(x_f) + \frac{\vec{v}_f  \times \vec{B}(x^{n}_p)}{c} \right) 
\end{align}$$
Futre Position und velocity will be set at Half Point:
$$ \begin{align}
           \mathbf{v}_f =\mathbf{v}_{\theta} \quad \quad \theta=0.5 \\
          \vec{\mathbf{v}}^{n+1}=\frac{\vec{\mathbf{v}}_f - \vec{\mathbf{v}}^n}{\theta}+\vec{\mathbf{v}}^n           
\end{align}$$
$$
\begin{align}
\mathbf{v}_f=\mathbf{v}_p^n+ \theta \frac{q_p \Delta t}{ m_p}\left(\mathbf{E}_p^{n+\theta}+ \frac{\mathbf{v}_f \times \mathbf{B}_p^n}{c}\right) \quad | \times B \quad , \quad \cdot B \\
\end{align}$$
$$
\begin{align}
\mathbf{v}_f \times \mathbf{B}_p^n=\mathbf{v}_p^n \times \mathbf{B}_p^n+\frac{q_p \theta \Delta t}{ m_p}\left(\mathbf{E}_p^{n+\theta} \times \mathbf{B}_p^n + \frac{\mathbf{v}_f \times \mathbf{B}_p^n}{c} \times \mathbf{B}_p^n\right)\\
\mathbf{v}_f \cdot \mathbf{B}_p^n=\mathbf{v}_p^n \cdot \mathbf{B}_p^n+\frac{q_p \Delta t \theta}{m_p}\left(\mathbf{E}_p^{n+\theta} \cdot \mathbf{B}_p^n + \frac{\mathbf{v}_f \times \mathbf{B}_p^n}{c} \cdot \mathbf{B}_p^n\right)
\end{align}
$$ Bei der zweiten Gleichung fällt der zweite Term per Definition weg und damit $\left(\mathbf{v}_f \times \mathbf{B}_p^n\right) \times \mathbf{B}_p^n=-\mathbf{v}_f\left|B_p^{n}\right|^2+\left(\mathbf{v}_f \cdot \mathbf{B}_p^n\right) \cdot \mathbf{B}_p^n \quad
$ **Folgt:**
$$
\begin{align}
\mathbf{v}_f \times \mathbf{B}_p^n &= \mathbf{v}_p^n \times \mathbf{B}_p^n + \frac{q_p \Delta t \theta}{ m_p} \left( \mathbf{E}_p^{n+\theta} \times \mathbf{B}_p^n + \color{red}{(\mathbf{v}_f \cdot \mathbf{B}_p^n) \mathbf{B}_p^n} - \color{black}{\frac{\mathbf{v}_f |\mathbf{B}_p^n|^2}{c}} \right) \\
&= \mathbf{v}_p^n \times \mathbf{B}_p^n + \frac{q_p \Delta t \theta}{ m_p} \left( \mathbf{E}_p^{n+\theta} \times \mathbf{B}_p^n + \color{red}{\frac{\mathbf{v}_p^n \cdot \mathbf{B}_p^n}{c} \mathbf{B}_p^n + \frac{q_p \Delta t \theta}{ m_p c} \left( \mathbf{E}_p^{n+\theta} \cdot \mathbf{B}_p^n \right) \mathbf{B}_p^n} - \color{black}{\frac{\mathbf{v}_f |\mathbf{B}_p^n|^2}{c}} \right)
\end{align}
$$
Reuse Velocity and define:
$$
\hat{\mathbf{v}}_p=\mathbf{v}_p^n+\frac{q_p \Delta t \theta}{ m_p} \mathbf{E}_p^{n+\theta}
$$
this leads to this Equation:
$$
\begin{align}
\mathbf{v}_f \times \mathbf{B}_p^n = \hat{\mathbf{v}}_p \times \mathbf{B}_p^n + \frac{q_p \Delta t \theta}{ m_p c} (\hat{\mathbf{v}}_p \cdot \mathbf{B}_p^n ) \mathbf{B}_p^n  - \frac{\mathbf{v}_f |\mathbf{B}_p^n|^2}{c}
\end{align}
$$
Durch Umstellen nach $\mathbf{v}_f$ erhält. herleitung etwas lengthy Thats why skip:
$$
\boxed{
\rule{0pt}{3ex}
\mathbf{v}_f=\frac{\hat{\mathbf{v}}_p+\frac{q_p \Delta t \theta}{ m_p c}\left(\hat{\mathbf{v}}_p \times \mathbf{B}_p^n+\frac{q_p \Delta t \theta}{ m_p c}\left(\hat{\mathbf{v}}_p \cdot \mathbf{B}_p^n\right) \mathbf{B}_p^n\right)}{1+\frac{q_p^2 \Delta t^2 \theta^2}{ m_p^2 c^2}\left|B_p^n\right|^2}
\rule[-1.5ex]{0pt}{3ex}
}
$$
**$\mathbf{v_f}$ $\theta$ is set to 0.5**

# Derivation of Linearized Electromagnetic Field Equations

Starting with the given time-stepping equations for the electromagnetic fields:
$$
\begin{aligned}
& \mathbf{B}^{n+1} = \mathbf{B}^n - c \Delta t \nabla \times \mathbf{E}^{n+\theta}, \\
& \mathbf{E}^{n+1} = \mathbf{E}^n + c \Delta t \left( \nabla \times \mathbf{B}^{n+\theta} - \frac{4 \pi}{c} \mathbf{J}^{n+\theta} \right),
\end{aligned}
$$
we apply the linear interpolation for the fields at the intermediate time step $n+\theta$:
 $\mathbf{E}$, $\mathbf{B}$. Substituting into the equations, we get:
$$
\begin{aligned}
& \mathbf{E}^{n+\theta} = (1 - \theta) \mathbf{E}^n + \theta \mathbf{E}^{n+1}, \\
& \mathbf{B}^{n+\theta} = (1 - \theta) \mathbf{B}^n + \theta \mathbf{B}^{n+1}, \\
\end{aligned}
$$
Than we plug in
$$
\begin{aligned}
& \mathbf{E}^{n+\theta} = (1 -  \textcolor{green}{\theta) \mathbf{E}^n} +  \textcolor{green}{\theta \mathbf{E}^n} +\theta c \Delta t \left( \nabla \times \mathbf{B}^{n+\theta} - \frac{4 \pi}{c} \mathbf{J}^{n+\theta} \right) \\
& \textcolor{red}{\nabla \times} \mathbf{B}^{n+\theta}
= \textcolor{red}{\nabla \times} (1 - \textcolor{green}{\theta) \mathbf{B}^n}
+ \textcolor{red}{\nabla \times} \textcolor{green}{\theta \mathbf{B}^n}
- \textcolor{red}{\nabla \times} \theta c \Delta t 
\nabla \times \mathbf{E}^{n+\theta} \\
&\nabla \times(\nabla \times \mathbf{E})=\nabla(\nabla \cdot \mathbf{E})−\nabla^2\mathbf{E}=4\pi\nabla(\mathbf{\rho})−\nabla^2\mathbf{E}
\end{aligned}
$$
Green Terms Cancel and B gets plugged in in E
$$
\boxed{
\rule{0pt}{3ex}
\mathbf{E}^{n+\theta} 
- (c\theta\Delta t)^2 \nabla^2 \mathbf{E}^{n+\theta}
= 
\mathbf{E}^n 
+ c\theta\Delta t \left( \nabla \times \mathbf{B}^n - \frac{4\pi}{c} \mathbf{J}^{n+\theta} \right)
- (c\theta\Delta t)^2 4\pi \nabla \rho^{n+\theta}
\rule[-1.5ex]{0pt}{3ex}
}
$$
# Derivation of Evolved Moments at $\theta$

Here, $i$ denotes the index of the spatial grid.

$$
\begin{aligned}
\rho^{n}_i &= \sum_p q_p S\!\left(\mathbf{x}_i - \mathbf{x}_p^{n}\right), \\
\mathbf{J}^{n} &= \sum_p q_p \mathbf{v}_p S\!\left(\mathbf{x} - \mathbf{x}_p^{n}\right).
\end{aligned}
$$
**Taylor expansion of $\rho$ around $\mathbf{x}^n$:**
$$
\begin{align}
&\rho(x^{n+\theta})=\rho^{n}+(x^{n+\theta}-x^{n})\nabla \sum_p q_p S(x-x^{n}) \\
&(x^{n+\theta}-x^{n})=-\theta  \mathbf{v}_p \Delta t \\
&\rho^{n+\theta}=\rho^{n}-\theta \Delta t \nabla \cdot J^{n+\theta}
\end{align}
$$
**First Rewrite the Velocity in A Shorter Form:**
The the previous solved velocity will be put in: $
\mathbf{v}_f=\frac{\hat{\mathbf{v}}_p+\frac{q_p \Delta t}{2 m_p c}\left(\hat{\mathbf{v}}_p \times \mathbf{B}_p^n+\frac{q_p \Delta t}{2 m_p c}\left(\hat{\mathbf{v}}_p \cdot \mathbf{B}_p^n\right) \mathbf{B}_p^n\right)}{1+\frac{q_p^2 \Delta t^2}{4 m_p^2 c^2}\left|B_p^n\right|^2}$
$$
\begin{aligned}
&\mathbf{v}_f = R_p(\hat{\mathbf{v}}_p) \quad \quad \beta_p=\frac{q_p \Delta t}{2 m_p} \quad \quad \hat{\mathbf{v}}_p=\mathbf{v}_p^n+\beta_p \mathbf{E}_p^{n+\theta} \\
&R_p(\vec{k}) =\frac{\vec{k}+\frac{\beta_p}{ c} \vec{k} \times \mathbf{B}_p^n+\frac{\beta^2_p}{ c^2}\left(\vec{k} \cdot \mathbf{B}_p^n\right) \mathbf{B}_p^n}{1+\frac{\beta_p^2}{ c^2}\left|B_p^n\right|^2}\\
&\mathbf{v}_f = R_p( \mathbf{v}_p^n ) + \beta_p R_p ( \mathbf{E}_p^{n+\theta})
\end{aligned}
$$
**Now with the last eqaution Taylor also for  J:**
$$
\mathbf{J}^{n+\theta}=\sum_p q_p \mathbf{v}_{f,p} S\left(\mathbf{x}-\mathbf{x}^n\right)-\theta \Delta t \nabla \cdot \sum_p q_p |\mathbf{v}_f |^2_p \; S\left(\mathbf{x}-\mathbf{x}^n\right)
$$
For The Velocity you get $\hat{\mathbf{J}}^n$ and the mixed Terms with E are sorted behind and the red Terms are approx as 0.:
$$
\begin{aligned}
&\hat{\mathbf{J}}^n=\sum_p q_p R_p\left(\mathbf{v}_p\right)  S\left(\mathbf{x}-\mathbf{x}^n\right)-\theta \Delta t \nabla \cdot \sum_p q_p |R_p\left(\mathbf{v}_p\right)|^2  S\left(\mathbf{x}-\mathbf{x}^n\right)\\
&\mathbf{J}^{n+\theta}=\hat{\mathbf{J}}^n+\sum_p q_p  \beta_p R_p\left(\mathbf{E}^{n+\theta}\right) \; S\left(\mathbf{x}-\mathbf{x}^n\right)\color{red}{ -\theta \Delta t \nabla \cdot \sum_p q_p \beta_p R_p\left(\mathbf{E}^{n+\theta}\right) \; R_p (\mathbf{v}_p) S\left(\mathbf{x}-\mathbf{x}^n\right)- \theta \Delta t \nabla \sum_p q_p    \left (\beta_p R_p(\mathbf{E}^{n+\theta}) \right)^2 } \\
\end{aligned}
$$

So the final equation becomes:
$$
\begin{aligned}
\mathbf{J}^{n+\theta}
&= \hat{\mathbf{J}}^n 
+ \sum_p q_p \, \beta_p \, R_p\!\bigl(\mathbf{E}^{n+\theta}\bigr) \, S\!\bigl(\mathbf{x} - \mathbf{x}^n\bigr) 
\\[1.2ex]
\rho^{n+\theta}
&= \rho^n - \theta \Delta t \, \nabla \cdot \mathbf{J}^{n+\theta}
\\[1.2ex]
\end{aligned}
$$
Einführung von Helfer Variable:
$$
\begin{aligned}
\hat{\rho}^n 
&= \rho^n - \theta \Delta t \, \nabla \cdot \hat{\mathbf{J}}^n
\\[1.2ex]
\rho^{n+\theta}
&= \hat{\rho}^n 
- \theta \Delta t \, \nabla \cdot \sum_p q_p \, \beta_p \, R_p\!\bigl(\mathbf{E}^{n+\theta}\bigr) \, S\!\bigl(\mathbf{x} - \mathbf{x}^n\bigr)
\end{aligned}
$$
Als Ernnierung die Gleichung die gelöst werden muss
$$
\begin{aligned}
\mathbf{E}^{n+\theta}
&- (c \theta \Delta t)^2 \, \nabla^2 \mathbf{E}^{n+\theta}
= \mathbf{E}^n 
+ c\theta \Delta t \!\left( \nabla \times \mathbf{B}^n 
- \frac{4\pi}{c} \mathbf{J}^{n+\theta} \right)
- (c \theta \Delta t)^2 4\pi \nabla \rho^{n+\theta}
\end{aligned}
$$
Abhänig von  $\mathbf{E}^{n+\theta}$ kommen nun auf der  rechte seite folgende Terme hinzu und werden danach zur linken SEite sortiert:
$$
\begin{aligned}
\mathrm{RHS} 
&= 
-(c\theta \Delta t) \, \frac{4\pi}{c} 
\sum_p q_p \, \beta_p \, R_p\!\bigl(\mathbf{E}^{n+\theta}\bigr) \, S\!\bigl(\mathbf{x} - \mathbf{x}^n\bigr)
\\[1.2ex]
&\quad
+ (c\theta \Delta t)^2 4\pi \, \nabla \!\biggl[ 
\theta \Delta t \, \nabla \cdot 
\sum_p q_p \, \beta_p \, R_p\!\bigl(\mathbf{E}^{n+\theta}\bigr) \, S\!\bigl(\mathbf{x} - \mathbf{x}^n\bigr)
\biggr]
\\[2ex]
\mu(\mathbf{k}) 
&= 4\pi \theta \Delta t 
\sum_p q_p \, \beta_p \, R_p(\mathbf{k}) \, S\!\bigl(\mathbf{x} - \mathbf{x}^n\bigr)
\\[2ex]
\mathrm{RHS}
&= -\mu\!\bigl(\mathbf{E}^{n+\theta}\bigr) 
+ (c\theta \Delta t)^2 \, \nabla \nabla \cdot \mu\!\bigl(\mathbf{E}^{n+\theta}\bigr)
\end{aligned}
$$
Nach dem alles Sortiert ist bekommt man dann:
$$
\begin{aligned}
 \mathbf{E}^{n+\theta} + \mu( \mathbf{E}^{n+\theta})
- (c\theta\Delta t)^2 
\left[
\nabla^2 \mathbf{E}^{n+\theta}
+ \nabla \nabla \cdot (\mu \cdot \mathbf{E}^{n+\theta})
\right]   = \mathbf{E}^n
+ c\theta\Delta t \left( \nabla \times \mathbf{B}^n - \frac{4\pi}{c} \hat{\mathbf{J}}^n \right)
- (c\theta\Delta t)^2 4\pi \nabla \hat{\rho}^n \\
\end{aligned}
$$

## 📚 References

**ChatGPT**
- Vu, H. X., & Brackbill, J. U. (1982). A numerical solution method for the two-dimensional magnetohydrodynamic equations. *Journal of Computational Physics*.
- Lapenta, G. (2006). Particle simulations of space weather. *Journal of Computational Physics*.

**Own Literature**
- Ott, Tobias & Pfeiffer, Marcel. (2023). PIC schemes for multi-scale plasma simulations. 10.13009/EUCASS2023-770.


In [6]:
import numpy as np 
arr1 = np.array([[1, 2], [3, 4]])
arr2 = np.array([[5, 6], [7, 8]])
"""
Concatenate Verknüpfen: erst über arr1 dann pber arr2 Iterieren
Achsen bestimmt die Ausgangs Achse arr[axis=0][axis=0] 
Axis 0
Mapping Elemente of arr1[index] und speicher in new[index] => Einfach aneinanderhängen
Axis 1
Mapping Elemente of arr1[:][index] und speicher in new[:][index] => Elemente werden nebeinander ANgehängt 
"""
arr = np.concatenate((arr1, arr2), axis=1)
print(arr)
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])
""""
Stacking fügt einfach nur anneinander und zusammenzufügen 
Axis 0
Takes arr1 und put it  in new[index] => Zusammenspeichern getrennt
Axis 1
Mapping Elemente of arr1 und speicher in new[index][0] für array2 index=1 
"""
arr = np.stack((arr1, arr2), axis=1)
print(arr)

[[1 2 5 6]
 [3 4 7 8]]
[[1 4]
 [2 5]
 [3 6]]


In [7]:
arr1 = np.zeros([3,5])
print(np.shape(arr1))
arr1[:,2]=[1,1,1]
arr1[:,3]=[7,0,7]
result = np.sum(arr1**2, axis=0)  # Summe der Quadrate entlang axis=0
print(np.shape(result))  # Ausgabe: (5,)
print(result)

(3, 5)
(5,)
[ 0.  0.  3. 98.  0.]


In [8]:
dim=(5,4,None)
J_hat = np.zeros([3 if False else 1, *dim])
print(np.shape(J_hat))


TypeError: 'NoneType' object cannot be interpreted as an integer

https://github.com/FunPhantasie/SemesterProject